# System for access segmentation

##### imports

In [180]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

#### Get data from results/data folder

In [181]:
#data = pd.data = pd.read_excel("../server/public/data/User-624921c5e1c4d602ddd19695-1649206670127.xlsx", skiprows=1)

In [182]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [183]:
data = pd.read_excel("/content/drive/MyDrive/User-624921c5e1c4d602ddd19695-1649206670127.xlsx", skiprows=1)

In [184]:
data

,user_id,position,org_structure,access_name
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:GOOGLE:ACCOUNT
1,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BAMP900:PB:USER
2,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BAMP900:RMS:HR_USER
3,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BGWP960:PB:FIORI_FLP
4,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BAMP900:FI:TV:TRAVELER
...,...,...,...,...
199530,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BAMP900:HR:ESS:CLAIMS_REIMBURSEMENT
199531,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BGWP960:IT:GW:PASSWORD_SELF_SERVICE
199532,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS
199533,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING


In [185]:
data.columns = ["user_id", "job", "department", "privileges_data"]

In [186]:
data = data.dropna()

In [187]:
data

,user_id,job,department,privileges_data
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:GOOGLE:ACCOUNT
1,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BAMP900:PB:USER
2,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BAMP900:RMS:HR_USER
3,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BGWP960:PB:FIORI_FLP
4,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",PRIV:ROLE:BAMP900:FI:TV:TRAVELER
...,...,...,...,...
199530,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BAMP900:HR:ESS:CLAIMS_REIMBURSEMENT
199531,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BGWP960:IT:GW:PASSWORD_SELF_SERVICE
199532,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS
199533,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,PRIV:ROLE:BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING


In [188]:
user_data = data.drop("privileges_data", axis=1)

In [189]:
user_data

,user_id,job,department
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"
1,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"
2,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"
3,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"
4,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"
...,...,...,...
199530,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS
199531,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS
199532,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS
199533,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS


In [190]:
privileges_data = pd.DataFrame(data["privileges_data"])

In [191]:
privileges_data.privileges_data = privileges_data["privileges_data"].str[10:]

I saw anomaly in data where in some of the records there is ::(double) symbols so i need to remove it so the data is clear

In [192]:
for indx in privileges_data[privileges_data["privileges_data"].str.startswith(":")].index.tolist():
    privileges_data.loc[indx].privileges_data = privileges_data.loc[indx].privileges_data[1:]

In [193]:
len(privileges_data.loc[privileges_data["privileges_data"].str.startswith(":")].privileges_data)

0

In [194]:
privileges_data

,privileges_data
0,GOOGLE:ACCOUNT
1,BAMP900:PB:USER
2,BAMP900:RMS:HR_USER
3,BGWP960:PB:FIORI_FLP
4,BAMP900:FI:TV:TRAVELER
...,...
199530,BAMP900:HR:ESS:CLAIMS_REIMBURSEMENT
199531,BGWP960:IT:GW:PASSWORD_SELF_SERVICE
199532,BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS
199533,BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING


In [195]:
privileges_data['priv'] = privileges_data['privileges_data']
privileges_data['system_name'] = privileges_data['privileges_data'].str.split(':', 1, expand=True)[0]

In [196]:
privileges_data = privileges_data.drop('privileges_data', axis=1)

In [197]:
privileges_data

,priv,system_name
0,GOOGLE:ACCOUNT,GOOGLE
1,BAMP900:PB:USER,BAMP900
2,BAMP900:RMS:HR_USER,BAMP900
3,BGWP960:PB:FIORI_FLP,BGWP960
4,BAMP900:FI:TV:TRAVELER,BAMP900
...,...,...
199530,BAMP900:HR:ESS:CLAIMS_REIMBURSEMENT,BAMP900
199531,BGWP960:IT:GW:PASSWORD_SELF_SERVICE,BGWP960
199532,BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS,BAMP900
199533,BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING,BGWP960


In [198]:
user_data['system_name'] = privileges_data['system_name']
user_data['priv'] = privileges_data['priv']

In [199]:
job="POSTDOCTORAL FELLOW"
department="BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"

segmentated_data = user_data.where((user_data.job == job) & (user_data.department == department))
segmentated_data.dropna(inplace=True)

In [200]:
segmentated_data

,user_id,job,department,system_name,priv
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",GOOGLE,GOOGLE:ACCOUNT
1,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BAMP900,BAMP900:PB:USER
2,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BAMP900,BAMP900:RMS:HR_USER
3,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BGWP960,BGWP960:PB:FIORI_FLP
4,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BAMP900,BAMP900:FI:TV:TRAVELER
...,...,...,...,...,...
198873,ZIAA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BGWP960,BGWP960:HR:GW:CLAIMS_TRAVEL_OLD_TIL
198874,ZIAA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BGWP960,BGWP960:IT:GW:PASSWORD_SELF_SERVICE
198875,ZIAA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BAMP900,BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS
198876,ZIAA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BGWP960,BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING


In [201]:
avarege_count = int(segmentated_data.priv.value_counts().sum() / len(segmentated_data.priv.value_counts()))
count = 0
for p_c in segmentated_data.priv.value_counts():
    if p_c < avarege_count:
        break
    count += 1
priv_list = segmentated_data.priv.value_counts().index.tolist()[:count]

In [202]:
priv_list

['BGRP940:KAUST:END_USERS',
 'BAMP900:HR:ESS:CLAIMS_REIMBURSEMENT',
 'SMP600:SM:ITSM:SELF_SERVICE',
 'BGRP940:GRAC:END_USERS_BASE',
 'BGWP960:IT:FIORI_INBOX_APPROVAL',
 'BGWP960:IT:GW:KAUST_ACCES_REQUEST',
 'BGWP960:GA:GASC_MYREQUEST',
 'SMP600:SM:ITSM:CREATE_INTERACTION',
 'BGWP960:KAUST:END_USERS',
 'BGRP940:GRAC:BUSINESS_ENDUSERS',
 'SMP600:KAUST:END_USERS',
 'BGWP960:IT:GW:END_USER',
 'BGWP960:IT:GW:PASSWORD_SELF_SERVICE',
 'BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING',
 'BGWP960:KS:KITS_EMPLOYEE_EQUIPMENT_GRP',
 'BAMP900:RMS:HR_USER',
 'BGWP960:KS:KITS_EMAIL_SERVICES',
 'BGWP960:KS:KITS_SECURITY_SERVICES',
 'BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS',
 'BAMP900:PB:USER',
 'GOOGLE:ACCOUNT',
 'BAMP900:HR:ESS:EMP_SELF_SERVICE',
 'BGWP960:HR:GW:EMP_SELF_SERVICE',
 'BGWP960:GA:GASC_REQUESTER',
 'BAMP900:SLCM:RM_END_USER',
 'BAMP900:KAUST:END_USERS',
 'BAMP900:RMS:TV:HR_USER',
 'BAMP900:IT:UWL:ENDUSER',
 'BAMP900:FI:TV:TRAVELER',
 'BGWP960:PB:FIORI_FLP',
 'BGWP960:IT:GW:USER_NOTIF',
 'BGWP960:I

### Get information about dataset properties

#### Get columns

In [203]:
user_data.columns

Index(['user_id', 'job', 'department', 'system_name', 'priv'], dtype='object')

#### Get unique attributes in every column

In [204]:
for c in user_data.columns:
    print(c, len(user_data[c].value_counts()))

user_id 6784
job 1581
department 463
system_name 9
priv 2122


#### Get unique privileges count for every system 

In [205]:
system_privs_count = 0
for s in user_data.system_name.unique():
    print(s, len(user_data[user_data.system_name == s].priv.unique()))
    system_privs_count += len(user_data[user_data.system_name == s].priv.unique())

print("Uniqua privs for every system count = ", system_privs_count)

GOOGLE 1
BAMP900 1751
BGWP960 71
SMP600 22
BGRP940 7
EMP980 57
BMP950 210
BCMP970 2
IT 1
Uniqua privs for every system count =  2122


#### get how many times system occurs in data

In [206]:
user_data.system_name.value_counts()

BGWP960    79886
BAMP900    77340
SMP600     16348
BGRP940    14272
EMP980      5542
BMP950      3642
GOOGLE      2406
BCMP970       66
IT             9
Name: system_name, dtype: int64

We can conclude that BCMP970 and IT systems contains special privileges for just one certain department and job

#### get data about privileges. Are there privileges which are the same for various systems

In [207]:
system_privs_count - len(user_data.priv.unique())

0

### Prepare data for clustering

In simple words we don't need data for job and department to cluster data, because clustering is performed on privileges for all users.

So we will create new table with user id - user_id, and all of the privileges as attributes with values of 0 or 1. 0 - don't have that role, 1 - have that role.

In [208]:
user_privs_data = pd.DataFrame()

Create all privileges(categorical data) compatabel for clustering by transforming all privs into numerical data.

In [209]:
priv_dummies = pd.DataFrame()

In [210]:
priv_dummies['user_id'] = user_data.user_id

In [211]:
priv_dummies = pd.concat([priv_dummies,pd.get_dummies(user_data.priv)], axis=1)

In [212]:
user_privs_data = priv_dummies.groupby('user_id').sum().reset_index()

In [213]:
user_privs_data

,user_id,BAMP900:AIS:AUDIT_ADMINISTRATOR,BAMP900:AIS:BA:CON_FIN_STATEMENTS,BAMP900:AIS:BA:INDIV_FIN_STATEMENTS,BAMP900:AIS:DATA_PROTECTION,BAMP900:AIS:INT:FINANCIAL_AUDIT,BAMP900:AIS:INTERNAL_AUDIT_DISPLAY_ALL,BAMP900:AIS:SYSTEMS_AUDIT,BAMP900:BI:HR:ANALYTICS_DEP_SEC_MAINT,BAMP900:BPM:GASC_AGENT,...,SMP600:SM:BRMS:STATUS_MANAGER,SMP600:SM:ITSM:CREATE_INTERACTION,SMP600:SM:ITSM:CREATE_SERVICE_REQUEST,SMP600:SM:ITSM:MANAGER,SMP600:SM:ITSM:SELF_SERVICE,SMP600:SM:ITSM:SERVICEDESK_AGENT,SMP600:SM:ITSM:SERVICE_TEAM,SMP600:SM:ITSM:SOLMANUI_CHANGE_MANAGM,SMP600:SM:ITSM:SOLMANUI_SER_DES_AGENT,SMP600:SM:ITSM:SOLMANUI_TEAM_MANAGER
0,AALISMNA,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,AAWARE,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,ABABTAM,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ABALAMP,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,ABASISN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6779,ZUPINF,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6780,ZURAYQOA,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6781,ZWAWIFM,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6782,ZYBINSA,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


In [214]:
agg_clustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=20)

In [215]:
fit_data = agg_clustering.fit(user_privs_data.drop('user_id', axis=1))

In [216]:
fit_data.n_clusters_

37

In [217]:
fit_data.n_features_in_

2122

In [218]:
fit_data.labels_

array([ 3, 12, 24, ..., 11, 17, 10])

In [219]:
fit_data.feature_names_in_.shape

(2122,)

In [220]:
result = pd.DataFrame(fit_data.labels_, columns=['id'])

In [221]:
result.id.value_counts()

12    955
2     849
3     596
4     545
30    406
7     353
6     325
20    308
8     232
10    225
17    219
11    195
32    164
34    146
28    138
14    136
24    124
22    119
26    117
13     96
23     92
21     52
29     49
18     37
1      37
5      36
19     31
25     31
16     31
31     29
15     21
9      20
35     19
27     18
0      17
33      8
36      8
Name: id, dtype: int64

### Get result based on clustered data


#### Prepare data for extracting privileges from given label

In [222]:
user_data

,user_id,job,department,system_name,priv
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",GOOGLE,GOOGLE:ACCOUNT
1,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BAMP900,BAMP900:PB:USER
2,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BAMP900,BAMP900:RMS:HR_USER
3,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BGWP960,BGWP960:PB:FIORI_FLP
4,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",BAMP900,BAMP900:FI:TV:TRAVELER
...,...,...,...,...,...
199530,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,BAMP900,BAMP900:HR:ESS:CLAIMS_REIMBURSEMENT
199531,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,BGWP960,BGWP960:IT:GW:PASSWORD_SELF_SERVICE
199532,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,BAMP900,BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS
199533,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,BGWP960,BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING


In [223]:
unique_user_data = user_data.groupby(['user_id', 'job', 'department']).describe().reset_index()

In [224]:
unique_user_data

user_id                                   job  \
                                                       
0     AALISMNA                   POSTDOCTORAL FELLOW   
1       AAWARE                               STUDENT   
2      ABABTAM                              ENGINEER   
3      ABALAMP  LAB SETUP AND MAINTENANCE TECHNICIAN   
4      ABASISN                DATA CENTER SPECIALIST   
...        ...                                   ...   
6779    ZUPINF      INTERNATIONAL AFFAIRS SPECIALIST   
6780  ZURAYQOA          NETWORK SYSTEM ENGINEER (L2)   
6781   ZWAWIFM                      BUSINESS MANAGER   
6782   ZYBINSA           SENIOR TECHNICAL SPECIALIST   
6783    weny0a                               STUDENT   

                                   department system_name                  \
                                                    count unique      top   
0     BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG          33      5  BGWP960   
1           EARTH SCIENCE & ENGINEERING (PHD)          25      4  BGWP960   
2                                        RPIC           3      1   EMP980   
3                          MARINE ENGINEERING          22      4  BAMP900   
4          DATA CENTERS & CRITICAL FACILITIES           6      2  BGWP960   
...                                       ...         ...    ...      ...   
6779       ADMISSIONS AND STUDENT RECRUITMENT          41      5  BGWP960   
6780                NETWORKS & COMMUNICATIONS          39      5  BAMP900   
6781                             KAUST HEALTH          34      4  BAMP900   
6782                               INORGANICS          35      5  BGWP960   
6783     MATERIALS SCIENCE & ENGINEERING (MS)          22      3  BGWP960   

           priv                                      
     freq count unique                     top freq  
0      16    33     33          GOOGLE:ACCOUNT    1  
1      13    25     25          GOOGLE:ACCOUNT    1  
2       3     3      3    EMP980:SRM:REQUESTOR    1  
3       9    22     22     BAMP900:RMS:HR_USER    1  
4       5     6      6  BGWP960:IT:GW:END_USER    1  
...   ...   ...    ...                     ...  ...  
6779   20    41     41         BAMP900:PB:USER    1  
6780   19    39     39     BAMP900:RMS:HR_USER    1  
6781   15    34     34         BAMP900:PB:USER    1  
6782   16    35     35         BAMP900:PB:USER    1  
6783   13    22     22          GOOGLE:ACCOUNT    1  

[6784 rows x 11 columns]

In [225]:
unique_user_data.drop(['system_name', 'priv'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [226]:
unique_user_data.columns = ['user_id', 'job', 'department']

In [227]:
unique_user_data

,user_id,job,department
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG"
1,AAWARE,STUDENT,EARTH SCIENCE & ENGINEERING (PHD)
2,ABABTAM,ENGINEER,RPIC
3,ABALAMP,LAB SETUP AND MAINTENANCE TECHNICIAN,MARINE ENGINEERING
4,ABASISN,DATA CENTER SPECIALIST,DATA CENTERS & CRITICAL FACILITIES
...,...,...,...
6779,ZUPINF,INTERNATIONAL AFFAIRS SPECIALIST,ADMISSIONS AND STUDENT RECRUITMENT
6780,ZURAYQOA,NETWORK SYSTEM ENGINEER (L2),NETWORKS & COMMUNICATIONS
6781,ZWAWIFM,BUSINESS MANAGER,KAUST HEALTH
6782,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS


In [228]:
result_table = pd.concat([unique_user_data, user_privs_data.drop('user_id', axis=1)], axis=1)

In [229]:
result_table

,user_id,job,department,BAMP900:AIS:AUDIT_ADMINISTRATOR,BAMP900:AIS:BA:CON_FIN_STATEMENTS,BAMP900:AIS:BA:INDIV_FIN_STATEMENTS,BAMP900:AIS:DATA_PROTECTION,BAMP900:AIS:INT:FINANCIAL_AUDIT,BAMP900:AIS:INTERNAL_AUDIT_DISPLAY_ALL,BAMP900:AIS:SYSTEMS_AUDIT,...,SMP600:SM:BRMS:STATUS_MANAGER,SMP600:SM:ITSM:CREATE_INTERACTION,SMP600:SM:ITSM:CREATE_SERVICE_REQUEST,SMP600:SM:ITSM:MANAGER,SMP600:SM:ITSM:SELF_SERVICE,SMP600:SM:ITSM:SERVICEDESK_AGENT,SMP600:SM:ITSM:SERVICE_TEAM,SMP600:SM:ITSM:SOLMANUI_CHANGE_MANAGM,SMP600:SM:ITSM:SOLMANUI_SER_DES_AGENT,SMP600:SM:ITSM:SOLMANUI_TEAM_MANAGER
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,AAWARE,STUDENT,EARTH SCIENCE & ENGINEERING (PHD),0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,ABABTAM,ENGINEER,RPIC,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ABALAMP,LAB SETUP AND MAINTENANCE TECHNICIAN,MARINE ENGINEERING,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,ABASISN,DATA CENTER SPECIALIST,DATA CENTERS & CRITICAL FACILITIES,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6779,ZUPINF,INTERNATIONAL AFFAIRS SPECIALIST,ADMISSIONS AND STUDENT RECRUITMENT,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6780,ZURAYQOA,NETWORK SYSTEM ENGINEER (L2),NETWORKS & COMMUNICATIONS,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6781,ZWAWIFM,BUSINESS MANAGER,KAUST HEALTH,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6782,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


In [230]:
result_table.insert(3, "cluster_label", result.id)

In [231]:
result_table

,user_id,job,department,cluster_label,BAMP900:AIS:AUDIT_ADMINISTRATOR,BAMP900:AIS:BA:CON_FIN_STATEMENTS,BAMP900:AIS:BA:INDIV_FIN_STATEMENTS,BAMP900:AIS:DATA_PROTECTION,BAMP900:AIS:INT:FINANCIAL_AUDIT,BAMP900:AIS:INTERNAL_AUDIT_DISPLAY_ALL,...,SMP600:SM:BRMS:STATUS_MANAGER,SMP600:SM:ITSM:CREATE_INTERACTION,SMP600:SM:ITSM:CREATE_SERVICE_REQUEST,SMP600:SM:ITSM:MANAGER,SMP600:SM:ITSM:SELF_SERVICE,SMP600:SM:ITSM:SERVICEDESK_AGENT,SMP600:SM:ITSM:SERVICE_TEAM,SMP600:SM:ITSM:SOLMANUI_CHANGE_MANAGM,SMP600:SM:ITSM:SOLMANUI_SER_DES_AGENT,SMP600:SM:ITSM:SOLMANUI_TEAM_MANAGER
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,AAWARE,STUDENT,EARTH SCIENCE & ENGINEERING (PHD),12,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,ABABTAM,ENGINEER,RPIC,24,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ABALAMP,LAB SETUP AND MAINTENANCE TECHNICIAN,MARINE ENGINEERING,4,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,ABASISN,DATA CENTER SPECIALIST,DATA CENTERS & CRITICAL FACILITIES,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6779,ZUPINF,INTERNATIONAL AFFAIRS SPECIALIST,ADMISSIONS AND STUDENT RECRUITMENT,14,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6780,ZURAYQOA,NETWORK SYSTEM ENGINEER (L2),NETWORKS & COMMUNICATIONS,4,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6781,ZWAWIFM,BUSINESS MANAGER,KAUST HEALTH,11,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
6782,ZYBINSA,SENIOR TECHNICAL SPECIALIST,INORGANICS,17,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


### Extract all data from clusters with selected job and department in them

In [232]:
result_data = result_table.where((result_table.job == job) & (result_table.department == department))

In [233]:
result_data.dropna(inplace=True)

In [234]:
result_data = result_data.reset_index()

In [235]:
result_data

,index,user_id,job,department,cluster_label,BAMP900:AIS:AUDIT_ADMINISTRATOR,BAMP900:AIS:BA:CON_FIN_STATEMENTS,BAMP900:AIS:BA:INDIV_FIN_STATEMENTS,BAMP900:AIS:DATA_PROTECTION,BAMP900:AIS:INT:FINANCIAL_AUDIT,...,SMP600:SM:BRMS:STATUS_MANAGER,SMP600:SM:ITSM:CREATE_INTERACTION,SMP600:SM:ITSM:CREATE_SERVICE_REQUEST,SMP600:SM:ITSM:MANAGER,SMP600:SM:ITSM:SELF_SERVICE,SMP600:SM:ITSM:SERVICEDESK_AGENT,SMP600:SM:ITSM:SERVICE_TEAM,SMP600:SM:ITSM:SOLMANUI_CHANGE_MANAGM,SMP600:SM:ITSM:SOLMANUI_SER_DES_AGENT,SMP600:SM:ITSM:SOLMANUI_TEAM_MANAGER
0,0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,118,ABROUKMD,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,496,ALFARDH,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,623,ALHATTDM,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1204,AMANR,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,6308,WANGY0R,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59,6401,XIAOT,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
60,6717,ZHENX0B,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
61,6736,ZHOUY0E,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",17.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [236]:
result_data.cluster_label.value_counts()

3.0     33
17.0    18
6.0      4
11.0     4
8.0      3
7.0      1
Name: cluster_label, dtype: int64

#### Get  the best fitted cluster

In [237]:
best_cluster_label = result_data.cluster_label.value_counts().index.tolist()[0]

In [238]:
best_cluster_label

3.0

In [239]:
current_cluster_label_data = result_table[result_table['cluster_label'] == best_cluster_label]

In [240]:
current_cluster_label_data.reset_index(inplace=True)

In [281]:
users_count = current_cluster_label_data.shape[0]

In [282]:
users_count

596

#### Get information about privileges from that cluster

In [241]:
if 'index' in current_cluster_label_data.columns:
  current_cluster_label_data = current_cluster_label_data.drop('index', axis=1)
current_cluster_label_data

,user_id,job,department,cluster_label,BAMP900:AIS:AUDIT_ADMINISTRATOR,BAMP900:AIS:BA:CON_FIN_STATEMENTS,BAMP900:AIS:BA:INDIV_FIN_STATEMENTS,BAMP900:AIS:DATA_PROTECTION,BAMP900:AIS:INT:FINANCIAL_AUDIT,BAMP900:AIS:INTERNAL_AUDIT_DISPLAY_ALL,...,SMP600:SM:BRMS:STATUS_MANAGER,SMP600:SM:ITSM:CREATE_INTERACTION,SMP600:SM:ITSM:CREATE_SERVICE_REQUEST,SMP600:SM:ITSM:MANAGER,SMP600:SM:ITSM:SELF_SERVICE,SMP600:SM:ITSM:SERVICEDESK_AGENT,SMP600:SM:ITSM:SERVICE_TEAM,SMP600:SM:ITSM:SOLMANUI_CHANGE_MANAGM,SMP600:SM:ITSM:SOLMANUI_SER_DES_AGENT,SMP600:SM:ITSM:SOLMANUI_TEAM_MANAGER
0,AALISMNA,POSTDOCTORAL FELLOW,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,ABDALIAM,CYBERSECURITY SPECIALIST,INFORMATION SECURITY,3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,ABDALLA,POST-DOCTORAL FELLOW,"COMPUTER, ELEC, MATH SCIENCES & ENG",3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,ABDEAM0A,POSTDOCTORAL FELLOW,"COMPUTER, ELEC, MATH SCIENCES & ENG",3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,ABDELAAA,POSTDOCTORAL FELLOW,ALI I.AL-NAIMI PETROLEUM ENGINEERING RC,3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,ZHAOW0B,POSTDOCTORAL FELLOW,CLEAN COMBUSTION RESEARCH CENTER,3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
592,ZHOUJ0E,RESEARCH TECHNICIAN,"BIOLOGICAL, ENVIRONMENTAL SCIENCE & ENG",3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
593,ZHOUY0B,POSTDOCTORAL FELLOW,DIVISION OF PHYSICAL SCIENCE AND ENG'G,3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
594,ZHUH0C,POSTDOCTORAL FELLOW,CATALYSIS RESEARCH CENTER,3,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0


In [268]:
privileges_counts = {}
for pr in current_cluster_label_data.columns[4:]:
  if 1 in current_cluster_label_data[pr].value_counts().index.tolist():
    privileges_counts[pr] = current_cluster_label_data[pr].value_counts()[1]

In [292]:
#privileges_counts = dict(sorted(privileges_counts.items(),key= lambda x:x[1]))
sorted_privileges_counts = sorted(privileges_counts.items(), key=lambda x: x[1], reverse=True)
filtered_privileges_counts = list(filter(lambda x: x[1] > users_count/3, sorted_privileges_counts))

#### Filter the most relevant privileges from that cluster

In [293]:
filtered_privileges_counts

[('BAMP900:FI:TV:TRAVELER', 596),
 ('BAMP900:HR:ESS:EMP_SELF_SERVICE', 596),
 ('BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS', 596),
 ('BAMP900:IT:UWL:ENDUSER', 596),
 ('BAMP900:PB:USER', 596),
 ('BAMP900:RMS:HR_USER', 596),
 ('BAMP900:RMS:TV:HR_USER', 596),
 ('BAMP900:SLCM:RM_END_USER', 596),
 ('BGWP960:GA:GASC_MYREQUEST', 596),
 ('BGWP960:GA:GASC_REQUESTER', 596),
 ('BGWP960:HR:GW:EMP_SELF_SERVICE', 596),
 ('BGWP960:IT:FIORI_INBOX_APPROVAL', 596),
 ('BGWP960:IT:GW:END_USER', 596),
 ('BGWP960:IT:GW:PASSWORD_SELF_SERVICE', 596),
 ('BGWP960:KAUST:END_USERS', 596),
 ('BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING', 596),
 ('BGWP960:KS:KITS_EMAIL_SERVICES', 596),
 ('BGWP960:KS:KITS_SECURITY_SERVICES', 596),
 ('BGWP960:KS:KITS_EMPLOYEE_EQUIPMENT_GRP', 595),
 ('BGWP960:PB:FIORI_FLP', 595),
 ('BGWP960:IT:GW:USER_WORKFLOW', 594),
 ('BGWP960:IT:GW:USER_REPORTS', 593),
 ('BAMP900:KAUST:END_USERS', 592),
 ('BGWP960:IT:GW:KAUST_ACCES_REQUEST', 592),
 ('BGWP960:IT:GW:USER_NOTIF', 592),
 ('SMP600:KAUST:END_USER

In [294]:
suggested_privileges = [v[0] for v in filtered_privileges_counts]

In [295]:
suggested_privileges

['BAMP900:FI:TV:TRAVELER',
 'BAMP900:HR:ESS:EMP_SELF_SERVICE',
 'BAMP900:IT:BUSINESS_JOB_RELEASE_ACCESS',
 'BAMP900:IT:UWL:ENDUSER',
 'BAMP900:PB:USER',
 'BAMP900:RMS:HR_USER',
 'BAMP900:RMS:TV:HR_USER',
 'BAMP900:SLCM:RM_END_USER',
 'BGWP960:GA:GASC_MYREQUEST',
 'BGWP960:GA:GASC_REQUESTER',
 'BGWP960:HR:GW:EMP_SELF_SERVICE',
 'BGWP960:IT:FIORI_INBOX_APPROVAL',
 'BGWP960:IT:GW:END_USER',
 'BGWP960:IT:GW:PASSWORD_SELF_SERVICE',
 'BGWP960:KAUST:END_USERS',
 'BGWP960:KS:KITS_CONFERENC_ROOM_BOOKING',
 'BGWP960:KS:KITS_EMAIL_SERVICES',
 'BGWP960:KS:KITS_SECURITY_SERVICES',
 'BGWP960:KS:KITS_EMPLOYEE_EQUIPMENT_GRP',
 'BGWP960:PB:FIORI_FLP',
 'BGWP960:IT:GW:USER_WORKFLOW',
 'BGWP960:IT:GW:USER_REPORTS',
 'BAMP900:KAUST:END_USERS',
 'BGWP960:IT:GW:KAUST_ACCES_REQUEST',
 'BGWP960:IT:GW:USER_NOTIF',
 'SMP600:KAUST:END_USERS',
 'BGRP940:GRAC:END_USERS_BASE',
 'BAMP900:HR:ESS:CLAIMS_REIMBURSEMENT',
 'BGRP940:GRAC:BUSINESS_ENDUSERS',
 'SMP600:SM:ITSM:CREATE_INTERACTION',
 'SMP600:SM:ITSM:SELF_SERVI

### Applying PCA to the model data

To reduce dimentionality of the dataset we are going to use PCA from sckitlearn

In [243]:
pca = PCA()

In [244]:
new_transformed_privs = pca.fit_transform(user_privs_data.drop('user_id', axis=1))

In [245]:
new_transformed_privs.shape

(6784, 2122)

In [246]:
transformed_user_privs_data = pd.DataFrame(new_transformed_privs, columns=user_privs_data.drop('user_id', axis=1).columns)

In [247]:
transformed_user_privs_data

,BAMP900:AIS:AUDIT_ADMINISTRATOR,BAMP900:AIS:BA:CON_FIN_STATEMENTS,BAMP900:AIS:BA:INDIV_FIN_STATEMENTS,BAMP900:AIS:DATA_PROTECTION,BAMP900:AIS:INT:FINANCIAL_AUDIT,BAMP900:AIS:INTERNAL_AUDIT_DISPLAY_ALL,BAMP900:AIS:SYSTEMS_AUDIT,BAMP900:BI:HR:ANALYTICS_DEP_SEC_MAINT,BAMP900:BPM:GASC_AGENT,BAMP900:CO:BO:COST_CENTER_REPORTS_DISP,...,SMP600:SM:BRMS:STATUS_MANAGER,SMP600:SM:ITSM:CREATE_INTERACTION,SMP600:SM:ITSM:CREATE_SERVICE_REQUEST,SMP600:SM:ITSM:MANAGER,SMP600:SM:ITSM:SELF_SERVICE,SMP600:SM:ITSM:SERVICEDESK_AGENT,SMP600:SM:ITSM:SERVICE_TEAM,SMP600:SM:ITSM:SOLMANUI_CHANGE_MANAGM,SMP600:SM:ITSM:SOLMANUI_SER_DES_AGENT,SMP600:SM:ITSM:SOLMANUI_TEAM_MANAGER
0,-1.759437,-0.121701,0.056221,-1.094561,-0.069786,0.566360,-0.671376,0.065018,0.057948,-0.192404,...,-2.299070e-17,3.368926e-17,2.200858e-17,-8.244743e-17,2.490649e-17,-3.881716e-17,2.676799e-17,-6.516696e-18,1.478637e-17,6.348643e-18
1,0.561862,-2.452957,0.225949,0.171170,0.071442,0.013606,0.046302,-0.004693,-0.048438,-0.207335,...,1.290730e-16,-4.243602e-17,4.658192e-17,-1.026023e-16,-1.108618e-16,8.694885e-17,-4.905929e-18,-6.417213e-18,-1.393000e-17,1.572332e-18
2,3.289687,1.470272,-1.358993,0.539196,-0.534905,-0.759480,-0.753971,0.356376,-0.252655,0.067599,...,-1.459561e-17,-4.622464e-17,-1.868790e-16,4.356610e-16,1.417473e-16,-2.885010e-17,-3.039655e-18,5.121428e-17,-4.241925e-17,-2.951627e-17
3,0.391382,1.247652,2.129600,-0.168786,0.117656,0.145089,0.530203,0.959199,-0.108366,0.125499,...,2.831349e-17,-2.305781e-16,6.508575e-18,-8.943181e-17,-3.576989e-17,7.518930e-17,5.598020e-17,5.216776e-17,7.021801e-18,-8.286099e-18
4,2.865644,0.775498,-0.339879,-0.224945,-0.306295,0.204290,-0.017125,0.035655,0.124051,0.406618,...,-4.743591e-17,1.480017e-16,-2.526764e-17,3.160856e-17,1.787597e-18,3.923232e-19,3.846437e-17,-4.488217e-17,-2.563209e-17,-1.421429e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6779,-2.261651,0.425020,-0.604698,-0.390523,-0.055208,0.236647,0.222787,-0.080432,-0.153938,-0.280694,...,-3.238282e-17,1.099555e-18,-5.903611e-18,-2.019728e-18,1.567524e-17,-1.029949e-17,2.168833e-17,1.532657e-17,1.344131e-17,-1.437272e-18
6780,0.175139,1.399501,1.824029,0.302817,0.392542,0.037501,1.203308,1.117520,-0.252794,0.069177,...,6.125910e-19,3.967323e-19,-1.149820e-18,-6.021854e-19,4.570082e-19,-9.201824e-19,-3.577718e-21,1.168320e-19,2.348685e-19,4.741325e-20
6781,-1.350201,0.787629,0.431626,-0.564037,-0.097030,0.889970,-0.606878,0.029399,0.282665,0.358901,...,-2.550804e-17,-1.594654e-17,-2.691435e-19,5.251229e-19,1.582531e-17,-2.652736e-17,-2.442128e-17,-2.187730e-17,3.701368e-18,2.898749e-19
6782,-2.005873,0.397232,-0.263735,-0.149273,-0.189591,-0.432439,-1.265418,0.399382,-0.217465,-0.098689,...,5.452739e-18,-5.762509e-18,9.975920e-19,1.367352e-18,4.392212e-18,-3.525138e-18,2.316790e-19,4.850511e-18,2.930461e-18,1.037171e-18


In [248]:
pca.explained_variance_ratio_.shape

(2122,)

In [249]:
user_privs_data.drop('user_id', axis=1).columns.shape

(2122,)

In [250]:
variance_explained = pd.DataFrame([pca.explained_variance_ratio_], columns=user_privs_data.drop('user_id', axis=1).columns)

In [251]:
variance_explained

,BAMP900:AIS:AUDIT_ADMINISTRATOR,BAMP900:AIS:BA:CON_FIN_STATEMENTS,BAMP900:AIS:BA:INDIV_FIN_STATEMENTS,BAMP900:AIS:DATA_PROTECTION,BAMP900:AIS:INT:FINANCIAL_AUDIT,BAMP900:AIS:INTERNAL_AUDIT_DISPLAY_ALL,BAMP900:AIS:SYSTEMS_AUDIT,BAMP900:BI:HR:ANALYTICS_DEP_SEC_MAINT,BAMP900:BPM:GASC_AGENT,BAMP900:CO:BO:COST_CENTER_REPORTS_DISP,...,SMP600:SM:BRMS:STATUS_MANAGER,SMP600:SM:ITSM:CREATE_INTERACTION,SMP600:SM:ITSM:CREATE_SERVICE_REQUEST,SMP600:SM:ITSM:MANAGER,SMP600:SM:ITSM:SELF_SERVICE,SMP600:SM:ITSM:SERVICEDESK_AGENT,SMP600:SM:ITSM:SERVICE_TEAM,SMP600:SM:ITSM:SOLMANUI_CHANGE_MANAGM,SMP600:SM:ITSM:SOLMANUI_SER_DES_AGENT,SMP600:SM:ITSM:SOLMANUI_TEAM_MANAGER
0,0.284145,0.163536,0.084364,0.046513,0.038757,0.026776,0.025488,0.017292,0.015471,0.012884,...,2.713932e-34,2.030493e-34,1.863008e-34,9.049346e-35,8.311576e-35,4.701204e-35,4.171235e-35,2.454215e-35,1.508762e-35,2.456512e-36


In [252]:
fit_pca_data = agg_clustering.fit(transformed_user_privs_data)

In [253]:
fit_pca_data.n_clusters_

36

In [254]:
result = pd.DataFrame(fit_pca_data.labels_, columns=['id'])

In [255]:
result

,id
0,3
1,12
2,24
3,4
4,5
...,...
6779,18
6780,4
6781,23
6782,0


In [256]:
result.id.value_counts()

12    955
5     849
3     595
4     536
30    406
7     362
0     350
18    329
11    327
10    229
9     225
23    195
32    164
34    146
24    131
28    131
14    119
13    105
27     95
21     92
22     50
29     49
8      37
1      37
6      36
25     31
16     31
19     31
31     29
20     21
15     21
35     19
26     18
2      17
33      8
17      8
Name: id, dtype: int64